<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Actualizados el 2023-2.</font>
</p>


# Ejercicios propuestos: Serialización
## Serialización

Los siguientes problemas se proveen como oportunidad de ejercitar los conceptos revisados en el material de **Serialización y manejo de bytes**. Si tienes dudas sobre algún problema o alguna solución, no dudes en dejar una *issue* en el [foro del curso](https://github.com/IIC2233/syllabus/issues).

### Ejercicio 1: Serialización con JSON  y *f-strings*

¡Han contaminado el recetario de las pizzas del Tini Tamburini! Es por esta razón que han venido a pedirte ayuda mediante este ejercicio propuesto.

Deberás deserializar el archivo **pizzeria.json** ubicado dentro de la carpeta `data`. Esta archivo contiene una lista de objetos de clase `Pizza`. Estos objetos tienen el atributo `ingredientes`, el cual ha sido contaminado con elementos que no son comestibles. Para identificarlos, el Tini le entrega un archivo llamado **comestibles.json** que tambien está dentro de la carpeta `data`, el cual será de ayuda para filtrar los ingredientes mediante el uso un `object_hook`. Finalmente, imprime todas las pizzas utilizando *f-strings*.

In [95]:
import json
import os


class Pizza:
    def __init__(self, nombre, ingredientes, *args, **kwargs):
        self.nombre = nombre
        self.ingredientes = ingredientes

    def __repr__(self):
        return self.nombre

    def __str__(self):
        return "Pizza: {} \nIngredientes:{}".format(self.nombre, self.ingredientes)


archivo_pizzas = open(os.path.join("data", "pizzeria.json"))
comestibles = open(os.path.join("data", "comestibles.json"))

lista_comestibles = json.load(comestibles)
lista_pizzas = json.load(archivo_pizzas)

archivo_pizzas.close()
comestibles.close()

for pizza in lista_pizzas:
    pizza = Pizza(nombre=pizza["nombre"], ingredientes=pizza["ingredientes"])
    ingredientes = list()
    for ingrediente in pizza.ingredientes:
        if ingrediente in lista_comestibles:
            ingredientes.append(ingrediente)
    pizza.ingredientes = ingredientes
    print(pizza)

# Carga las pizzas del archivo pizzeria.json que está dentro de data
# y finalmente imprime la lista de pizzas usando f-strings

Pizza: hawaiana 
Ingredientes:['pinia', 'queso', 'jamon']
Pizza: vegetariana 
Ingredientes:['queso', 'aceitunas', 'champiniones']
Pizza: machas a la parmesana 
Ingredientes:['machas', 'queso']
Pizza: pepperoni 
Ingredientes:['pepperoni', 'queso']
Pizza: pollo bbq 
Ingredientes:['cebolla', 'queso', 'pollo bbq']
Pizza: jamon palmito 
Ingredientes:['jamon', 'palmitos', 'queso']
Pizza: white 
Ingredientes:['aceitunas', 'tomate']
Pizza: napolitana 
Ingredientes:['queso', 'albahaca', 'tomate']
Pizza: queso 
Ingredientes:['queso', 'queso']


### Ejercicio 2: Serialización con JSON y *pickle*

Mientras estás de visita en un museo, se descubre que en las descripciones de un conjunto de obras de arte se encuentra un mensaje oculto, el cual solo se puede descifrar a través del uso de *pickle* para serialización. Es por esto, que el director del museo pide tu ayuda.

Para lograr descifrar el mensaje oculto, deberás implementar las siguientes funciones:

- `cargar_obras(ruta_obras)`: El archivo **operas.json** contiene toda la información de las obras. Las obras contienen algunos datos que no son relevantes para la solución de este misterio. Por esto, se te entrega el archivo **caratteristicas.json**, que está en la carpeta `data`, con las características que sí deberás obtener de las obras, mediante la función `obras_hook`, que deberá retornar una **lista de objetos de la clase `Obra`**.
- `generar_mensaje(lista_obras)`: Esta función debe serializar cada una de las obras utilizando `pickle`. Durante este proceso, deberás agregar a cada obra el attributo `messaggio`, con un mensaje generado por 3 palabras al azar del atributo `descrizione`. Los archivos generados deben ser guardados en la carpeta **Obras** que está dentro de la carpeta `data`. Cada archivo debe ser guardado con el siguiente formato: `<nome>-<autore>.opera` (sin `<` ni `>`), donde `nome` y `autore` son el nombre de la obra y el nombre de su respectivo autor. 

In [110]:
import json
import os
import pickle
import random


CARATTERISTICAS_CAMMINO = os.path.join("data", "caratteristicas.json")
OPERAS_CAMMINO = os.path.join("data", "operas.json")
# Camino := Path


class Obra:
    """
    Clase para las Obras de Antonini Da Ossa
    """

    def __init__(
        self,
        nome=None,
        autore=None,
        anno=None,
        posto=None,
        stile=None,
        descrizione=None,
    ):
        self.nome = nome
        self.autore = autore
        self.anno = anno
        self.posto = posto
        self.stile = stile
        self.descrizione = descrizione

    def __getstate__(self):
        """
        Serializa las obras, agregando el atributo 'messaggio' al diccionario.
        """
        nueva = self.__dict__.copy()

        # Split the description into words
        words = nueva.get("descrizione", "").split()

        # Check if there are enough words to generate a message
        if len(words) >= 3:
            # Randomly select three words
            random_words = random.sample(words, 3)
            nueva["messaggio"] = ' '.join(random_words)
        else:
            # Use the original description if there are fewer than 3 words
            nueva["messaggio"] = nueva.get("descrizione", "")

        return nueva
    
def cargar_obras(ruta_obras):
    """

    Funcion que carga las obras con las características pedidas,
    y luego entrega la lista de Obras.
    :param ruta_obras: Path de archivo de obras a cargar

    NOTA: DEBE USARSE la funcion obras_hook() como
     object hook para hacer el filtrado
    """
    return json.load(open(ruta_obras), object_hook=obras_hook)


def obras_hook(dict_obras):
    """
    Object hook that creates Obra objects and adds them to a list.
    Filters and extracts relevant characteristics from obras.
    """
    with open("data/caratteristicas.json") as f:
        characteristics = json.load(f)

    lista_obras = []

    for obra_key, obra_info in dict_obras.items():
        obra_attributes = {}
        for characteristic in characteristics:
            if characteristic in obra_info:
                obra_attributes[characteristic] = obra_info[characteristic]

        obra = Obra(**obra_attributes)
        lista_obras.append(obra)

    return lista_obras
    

def generar_mensaje(lista_obras):
    """
    Serializa las obras y las guarda en archivos
    :param lista_obras: Lista de objetos de tipo obra cargados.
    """
    pass


mi_obra = Obra(
    nome="Il Giovane del Christinno",
    autore="Fernandino Alfaro",
    anno=1502,
    stile="impressionismoArte astratta",
    posto="La Metropoli di Brunapoli",
    descrizione="Voluptatem sit quisquam aliquam modi numquam.Dolor amet quiquia voluptatem sit tempora etincidunt modi.",
)

serializado = pickle.dumps(mi_obra)
deserializado = pickle.loads(serializado)

cargar_obras(OPERAS_CAMMINO)

TypeError: argument of type 'int' is not iterable

### Ejercicio 3: Serialización usando Pickle y JSON

Eres conocid@ por tod@s por tu gran amor al cine, razón por la cual tu novia Victoria te ha pedido que por favor le recomiendes películas buenas para poder ver. Para hacer esta recomendación, debes hacer uso del archivo `movies.json` que está dentro de la carpeta `data`. 
Este archivo tiene un diccionario por película con varias características, de las cuales solamente debes guardar algunas. El procedimiento que debes llevar a cabo es el siguiente:

1. Completar la funcion `cargar_películas()` que debe deserializar la información de las películas del archivo `movies.json`, solamente incluyendo los datos que son relevantes especificados en el archivo `caracteristicas.pkl` (que está en la carpeta `data`), mediante la funcion `hook_peliculas()`. Esta última debe retorna una lista de objetos de la clase Pelicula. Notar que el archivo `caracteristicas.pkl` también debe ser deserializado, utilizando Pickle, para obtener la información en el.

2. Completar la función `recomendar_peliculas(lista_peliculas)` que recibe una lista de objetos de la clase Pelicula. Esta debe ser capaz de serializar, mediante Pickle, cada película en un archivo llamado `recomendaciones.pkl` (este lo crearás tu). Para la serialización, debes completar el método `__getstate__()` para poder agregar una opinión (tu recomendación) al diccionario a serializar de cada película. Notar que los strings de opinión a elegir se encuentran en el archivo `opiniones.pkl` que está dentro de `data`, por lo que debes deserializar este también, mediante Pickle, para obtenerlas. Puedes usar la librería `random` importada para elegir una opinión al azar.

In [ ]:
import json
import pickle
import random


class Pelicula:
    def __init__(
        self, title=None, category_name=None, release_year=None, running_time=None
    ):
        self.title = title
        self.category_name = category_name
        self.release_year = release_year
        self.running_time = running_time

    def __getstate__(self):
        """
        Aquí debes personalizar la serialización de las películas, agregando una opinion al
        diccionario. Recuerda obtener las opiniones deserializando el archivo opiniones.pkl
        con Pickle.
        """
        pass


def hook_peliculas(peliculas):
    """
    Debes utilizar esta función para personalizar la deserialización de las películas en
    movies.json.
    """
    pass


def cargar_peliculas():
    """
    Esta función debe deserializar la información de cada película, pero icluyendo solamente la
    información especificada en caracteristicas.pkl (este archivo debe ser deserializado con Pickle)
    utilizando la función hook_peliculas() para el filtrado.
    """
    pass


def recomendar_peliculas(peliculas):
    """
    Se deben serializar las películas en el archivo recomendaciones.pkl utilizando Pickle,
    incluyendo un mensaje de opinión mediante el método __getstate__() de la clase Pelicula.
    """
    pass


In [ ]:
# Main code
"""
Al correr este código, deberías obtener un archivo llamado recomendaciones.pkl con la información
de cada película y la opinion para cada una de estas.
"""
peliculas = cargar_peliculas()
recomendar_peliculas(peliculas)

In [ ]:
"""
El siguiente código puede ser corrido para comprobar que se realizo el ejercicio correctamente.
Deberían imprimirse diccionarios por película, con todas las caracteristicas de cada película del 
archivo caracteristicas.pkl como keys y una key "opinion" que tiene como value alguna de las tres 
opiniones del archivo opiniones.pkl.
"""

with open("recomendaciones.pkl", "rb") as file:
    peliculas_recomendadas = list()
    while True:
        try:
            peliculas_recomendadas.append(pickle.load(file))
        except EOFError:
            break
for p in peliculas_recomendadas:
    print(p.__dict__)

### Ejercicio 4: Serialización usando JSON

Durante las caóticas inscripciones de cursos para el segundo semestre del 2020 un hacker aprovechó la situación y corrompió la base de datos de los estudiantes, dejándolos con asignaturas inexistentes en el sistema. Afortunadamente, existe un archivo `asignaturas.json`, dentro de la carpeta `data`, en donde están los cursos disponibles. Tu trabajo será instanciar los Alumnos con los cursos correctos, filtrando (por medio de un hook) aquellos cursos que no correspondan y guardar todos los alumnos en un archivo llamado `alumnos.json`.

In [ ]:
import json
import os


class Alumno:
    def __init__(self, nombre, num_alumno, cursos):
        self.nombre = nombre
        self.num_alumno = num_alumno
        self.cursos = cursos

    def __repr__(self):
        string_cursos = ", ".join(self.cursos)
        return f"Asignaturas de {self.nombre}: \n{string_cursos}\n"


with open(os.path.join("data", "asignaturas.json")) as file:
    # Modifica la linea siguiente para cargar la LISTA de asignaturas
    asignaturas = None
    pass


def hook_function(diccionario):
    # Completa esta función para deserializar los alumnos.
    # Debes dejar solamente las asignaturas presentes en la lista cargada arriba
    pass


with open("alumnos.json", "r") as file:
    # Carga los alumnos en una lista usando object_hook
    lista_alumnos = None
    pass

print(*lista_alumnos, sep="\n")
